## Set Up

In [11]:
# loading in packages 
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import requests

In [2]:
# Read in file
df_iowa = pd.read_csv("Iowa_Liquor_Sales.csv",
                      usecols = ["Date", "Store Number", "City", "County", "Category Name", "Store Location",
                                 "Bottles Sold", "Sale (Dollars)", "Volume Sold (Liters)"])

In [3]:
# Merge with county dataset

# External dataset from: https://www.openintro.org/data/index.php?data=county
df_county = pd.read_csv("county.csv")
df_county = df_county[df_county["state"] == "Iowa"]
df_county["name"] = df_county["name"].str.removesuffix(" County").str.upper()
df_iowa = df_iowa.merge(df_county, left_on = "County", right_on = "name", how = "left")

# Drop all NA values 
df_iowa.dropna(inplace = True)
df_iowa.drop_duplicates(inplace = True)

## Feature Engineering

#### For Visualization Purposes:

In [4]:
# Lat and Long split for visualizations

df_iowa["Long"] = df_iowa["Store Location"].str.split(" ").str[1].str[1:]
df_iowa["Lat"] = df_iowa["Store Location"].str.split(" ").str[2].str[:-1]
df_iowa.drop("Store Location", axis = 1, inplace = True)

In [5]:
# Mapping drinks to more generic categories (ended up not using)

category_mapping = {
    "SCHNAPPS": "Schnapps",
    "CORDIALS & LIQUEURS": "Liqueurs",
    "WHISKIES": "Whiskey",
    "WHISKY": "Whiskey",
    "RUM": "Rum",
    "VODKA": "Vodka",
    "GIN": "Gin",
    "BRANDIES": "Brandy",
    "TEQUILA": "Tequila",
    "BOURBON": "Whiskey",
    "SCOTCH": "Whiskey",
    "MEZCAL": "Mezcal",
    "COCKTAILS": "Cocktails",
    "DISTILLED SPIRITS SPECIALTY": "Specialty Spirits",
    "CREME": "Liqueurs",
    "ANISETTE": "Liqueurs",
    "AMARETTO": "Liqueurs",
    "BEER": "Beer",
}

# Function to classify a specific category
def classify_category(category_name):
    for key, general_type in category_mapping.items():
        if key in category_name.upper():
            return general_type
    return "Other"

df_iowa['Category Name'] = df_iowa['Category Name'].apply(classify_category)

In [6]:
# Writing to .csv file for visualizations in r

df_iowa.to_csv("clean_df_no_na2.csv")

#### For Modeling Purposes:

In [7]:
# Using only important columns

df_iowa = df_iowa[["Sale (Dollars)", "Volume Sold (Liters)", "metro", "smoking_ban", "median_edu", "pop2017", "pop_change", "poverty", "homeownership", "multi_unit", "unemployment_rate", "per_capita_income", "median_hh_income"]]

In [8]:
# One hot encoding 
df_iowa["metro"] = pd.Categorical(df_iowa["metro"], categories=["no", "yes"], ordered = True) 
df_iowa["smoking_ban"] = pd.Categorical(df_iowa["smoking_ban"], categories=["none", "partial"], ordered = True)
df_iowa["median_edu"] = pd.Categorical(df_iowa["median_edu"], categories=["some_college", "hs_diploma", "bachelors"], ordered=True)

df_iowa = pd.get_dummies(df_iowa, drop_first = True)

# Getting colnames 
colnames = df_iowa.columns[2:].insert(0, 'intercept')

In [9]:
# Making 2 sets, one for DEAD and one for Booze R Us 

brus = dead = df_iowa.drop(["Volume Sold (Liters)", "Sale (Dollars)"], axis = 1)

# Different response variables
dol = df_iowa["Sale (Dollars)"]
vol = df_iowa["Volume Sold (Liters)"]

In [12]:
# Standardizing...

transformer = make_column_transformer((StandardScaler(), ["pop2017", "pop_change", "poverty", "homeownership", 
            "multi_unit", "unemployment_rate", "per_capita_income", "median_hh_income"]), remainder = "passthrough")

brus = transformer.fit_transform(brus)
dead = transformer.fit_transform(dead)

# Adding ones for intercept
ones_col = np.ones((brus.shape[0], 1))
brus =  np.hstack((ones_col, brus))
dead =  np.hstack((ones_col, dead))

## Booze R Us Models

In [13]:
# train/test split for brus

X_train, X_test, y_train, y_test = train_test_split(brus, dol, random_state = 403)

In [14]:
# LR for brus 

beta_brus_lr = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
brus_lr = pd.DataFrame(data={'x': colnames, 'betas': beta_brus_lr})
brus_lr.sort_values(by = "betas")

x       betas
12   median_edu_bachelors -109.448346
9               metro_yes  -21.099072
4           homeownership  -10.656670
1                 pop2017  -10.573284
6       unemployment_rate    2.478016
11  median_edu_hs_diploma    3.210429
10    smoking_ban_partial    7.437605
7       per_capita_income   12.458706
5              multi_unit   12.461273
2              pop_change   16.543584
8        median_hh_income   20.246583
3                 poverty   23.165358
0               intercept  183.921773

In [15]:
# Calculating MAE for BRUS LR 

y_pred_brus_lr = X_test @ beta_brus_lr
mae_brus_lr = np.mean(np.abs(y_test - y_pred_brus_lr))
mae_brus_lr

np.float64(143.43917949031206)

In [17]:
# RR for brus 

beta_brus_rr = np.linalg.inv(X_train.T @ X_train + 20 * np.identity(X_train.shape[1])) @ X_train.T @ y_train
brus_rr = pd.DataFrame(data={'x': colnames, 'betas': beta_brus_rr})
brus_rr.sort_values(by = "betas")

x       betas
12   median_edu_bachelors -109.418479
9               metro_yes  -21.090993
4           homeownership  -10.652880
1                 pop2017  -10.569771
6       unemployment_rate    2.479427
11  median_edu_hs_diploma    3.211159
10    smoking_ban_partial    7.438148
7       per_capita_income   12.457843
5              multi_unit   12.459041
2              pop_change   16.546465
8        median_hh_income   20.239166
3                 poverty   23.159915
0               intercept  183.913249

In [18]:
# Calculating MAE for BRUS RR

y_pred_brus_rr = X_test @ beta_brus_rr
mae_brus_rr = np.mean(np.abs(y_test - y_pred_brus_rr))
mae_brus_rr

np.float64(143.43895196502018)

## Models for DEAD

In [20]:
# train/test split for DEAD

X_train, X_test, y_train, y_test = train_test_split(dead, vol, random_state = 403)

In [21]:
# LR for dead

beta_dead_lr = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
dead_lr = pd.DataFrame(data={'x': colnames, 'betas': beta_dead_lr})
dead_lr.sort_values(by = "betas")

x      betas
12   median_edu_bachelors  -5.877793
9               metro_yes  -1.485982
1                 pop2017  -0.789829
4           homeownership  -0.076805
6       unemployment_rate   0.065027
10    smoking_ban_partial   0.065126
11  median_edu_hs_diploma   0.376854
2              pop_change   0.604002
8        median_hh_income   0.731127
7       per_capita_income   0.949607
3                 poverty   1.109084
5              multi_unit   1.234824
0               intercept  11.327155

In [22]:
# Calculating MAE for DEAD LR

y_pred_dead_lr = X_test @ beta_dead_lr
mae_dead_lr = np.mean(np.abs(y_test - y_pred_dead_lr))
mae_dead_lr

np.float64(8.497360097664282)

In [23]:
# RR for dead 

beta_dead_rr = np.linalg.inv(X_train.T @ X_train + 20 * np.identity(X_train.shape[1])) @ X_train.T @ y_train
dead_rr = pd.DataFrame(data={'x': colnames, 'betas': beta_dead_rr})
dead_rr.sort_values(by = "betas")

x      betas
12   median_edu_bachelors  -5.876172
9               metro_yes  -1.485481
1                 pop2017  -0.789642
4           homeownership  -0.076647
6       unemployment_rate   0.065107
10    smoking_ban_partial   0.065205
11  median_edu_hs_diploma   0.376888
2              pop_change   0.604166
8        median_hh_income   0.730718
7       per_capita_income   0.949564
3                 poverty   1.108781
5              multi_unit   1.234642
0               intercept  11.326645

In [24]:
# Calculating MAE for DEAD RR

y_pred_dead_rr = X_test @ beta_dead_rr
mae_dead_rr = np.mean(np.abs(y_test - y_pred_dead_rr))
mae_dead_rr

np.float64(8.497349547298615)